## Zaawansowany RAG z ewaluacją i pętlą

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

eval_prompt = ChatPromptTemplate.from_messages([
    ("system", "Oceń odpowiedź."),
    ("user", "Pytanie: {question}\nOdpowiedź: {answer}\nCzy odpowiedź odpowiada na pytanie? Odpowiedz tak/nie.")
])

def rag_with_eval(question, max_retries=2):
    for attempt in range(max_retries):
        context = format_docs(retriever.get_relevant_documents(question))
        answer = (prompt | llm | StrOutputParser()).invoke({"context": context, "question": question})
        eval_result = (eval_prompt | llm | StrOutputParser()).invoke({"question": question, "answer": answer})
        if "tak" in eval_result.lower():
            return f"✅ Odpowiedź zaakceptowana:\n{answer}"
        print("❌ Odpowiedź odrzucona, ponawiam próbę...")
    return "Nie udało się uzyskać poprawnej odpowiedzi."

print(rag_with_eval("Co to jest RAG?"))


### budowa VectorStore (FAISS) i Retrievera

In [ ]:
from langchain_community.vectorstores import FAISS

emb = OpenAIEmbeddings()  # wymaga OPENAI_API_KEY
vectorstore = FAISS.from_texts(splitted, embedding=emb)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

query = "Po co używa się retrievera?"
context = retriever.get_relevant_documents(query)
print("Znaleziony kontekst:")
for i, c in enumerate(context, 1):
    print(f"{i}.", c.page_content)

### prosty łańcuch RAG (prompt + kontekst + LLM)

In [ ]:
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Udziel precyzyjnej odpowiedzi wyłącznie na podstawie KONTEKSTU. Jeśli brak danych — powiedz, że nie wiesz."),
    ("system", "KONTEKST:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym jest FAISS i do czego służy?"))